In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Sequential


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.preprocessing import LabelEncoder

from PIL import Image 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [ ]:
data_set = "dogs-vs-cats"

import zipfile 
with zipfile.ZipFile("/kaggle/input/"+ data_set +"/train.zip","r") as z:
    z.extractall(".")
    # save all files to kaggle/files/images
    destination = '/kaggle/files/images/train'
    z.extractall(destination)
    
with zipfile.ZipFile("/kaggle/input/"+ data_set +"/test1.zip","r") as z:
    z.extractall(".")
    # save all files to kaggle/files/images
    destination = '/kaggle/files/images/test'
    z.extractall(destination)

In [ ]:
def list_full_paths(directory):
    return [os.path.join(directory, file) for file in os.listdir(directory)]

train = pd.DataFrame({'filepath': list_full_paths('/kaggle/files/images/train/train')})
train['truth_label'] = np.where(train['filepath'].str.contains('dog'), 'dog', 'cat')

test = pd.DataFrame({'filepath': list_full_paths('/kaggle/files/images/test/test1')})

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test = train_test_split(train, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale = 1./255,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    rotation_range=40,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    horizontal_flip=True,
                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_dataframe(dataframe=X_train, x_col='filepath', y_col='truth_label', class_mode='categorical', target_size = (64, 64), batch_size = 128)
test_set = test_datagen.flow_from_dataframe(dataframe=X_test, x_col='filepath', y_col='truth_label', class_mode='categorical', target_size = (64, 64), batch_size = 128)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


batches_augmented = train_datagen.flow_from_directory('/kaggle/files/images/', target_size = (512, 512), batch_size = 16, class_mode = 'categorical', seed=1234)
batches_real = test_datagen.flow_from_directory('/kaggle/files/images/', target_size = (512, 512), batch_size = 16, class_mode = 'categorical', seed=1234)

x_batch_augmented, y_batch_augmented = next(batches_augmented)
x_batch_real, y_batch_real = next(batches_real)

for i in range(16):
    image_augmented = x_batch_augmented[i]
    image_real = x_batch_real[i]
    
    title_add_on = "random image"
    if y_batch_augmented[i][1]: title_add_on =  "dog vs cat"

    plt.subplot(221)
    plt.imshow(image_real)
    plt.title("original " + title_add_on)

        
    plt.subplot(222)
    plt.imshow(image_augmented)
    plt.title("augmented " + title_add_on)

    plt.show()

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",
                padding="valid", input_shape = (64,64,3)))

classifier.add(MaxPooling2D(pool_size=2, strides=2, padding='valid'))

classifier.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu",
                padding="valid", input_shape = (64,64,3)))

classifier.add(MaxPooling2D(pool_size=2, strides=2, padding='valid'))

classifier.add(Flatten())
classifier.add(Dense(128,activation="relu")) 
classifier.add(Dense(2,activation="sigmoid")) 

In [ ]:
classifier.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = classifier.fit(training_set, validation_data = test_set, epochs=10)

In [ ]:
test_set2 = test_datagen.flow_from_dataframe(dataframe=test,
    directory = '/kaggle/files/images/test',
    x_col = 'filepath',
    y_col = None,
    class_mode = None,
    target_size = (64, 64),
    batch_size = 32,
    shuffle = False)

In [ ]:
test_preds = classifier.predict(test_set2, steps = np.ceil(test.shape[0] / 32))

test["test_preds"] = np.argmax(test_preds, axis = 1)
labels = dict((v,k) for k,v in training_set.class_indices.items())

test['test_preds'] = test['test_preds'].map(labels)

In [ ]:
sample_test = test.sample(64).reset_index(drop = True)

fig = plt.figure(1, figsize = (24, 20))
fig.suptitle("Sample Predictions")

for i in range(len(sample_test)):
    
    plt.subplot(10, 8, i + 1)
    image = load_img(sample_test.filepath[i])
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"Predicted as {sample_test['test_preds'][i]}")
    
plt.tight_layout()
plt.show()